In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer #Reemplazo de datos faltantes
import numpy as np

datos = pd.read_csv('D:/lvs/Master/ProyectoTFM/datos/Post-Procesing/df_tablaGN_metGA.csv')
datos

,fecha,PART_Media,PART_Mediana,PART_Max,PART_Min,dir,prec,presMax,presMin,racha,sol,tmax,tmed,tmin,velmedia,resultado
0,2017-01-01,42.972222,43.500000,119.000000,11.000000,99.000000,0.000000,962.200000,955.400000,3.600000,7.800000,13.500000,5.400000,-2.700000,0.600000,29.757484
1,2017-01-02,29.770277,28.513553,65.204259,13.267183,16.000000,0.000000,957.200000,952.700000,5.800000,8.800000,16.700000,7.400000,-2.000000,1.700000,29.757484
2,2017-01-03,29.770277,28.513553,65.204259,13.267183,11.000000,0.000000,961.100000,956.900000,3.600000,7.400000,16.500000,7.700000,-1.100000,0.800000,56.125000
3,2017-01-04,56.125000,61.500000,106.000000,11.000000,25.000000,0.000000,961.700000,958.000000,4.700000,4.200000,17.200000,9.000000,0.700000,0.600000,41.097902
4,2017-01-05,41.097902,36.000000,76.000000,18.000000,99.000000,0.000000,958.800000,954.600000,4.700000,6.100000,17.300000,9.600000,1.800000,0.300000,32.611111
5,2017-01-06,32.611111,35.500000,56.000000,11.000000,21.000000,0.000000,961.500000,957.600000,6.700000,9.100000,16.900000,7.800000,-1.200000,1.100000,37.840278
6,2017-01-07,37.840278,35.500000,104.000000,12.000000,99.000000,0.000000,963.100000,959.800000,2.200000,9.100000,16.400000,7.900000,-0.600000,0.600000,39.423611
7,2017-01-08,39.423611,40.000000,76.000000,12.000000,20.000000,0.000000,961.800000,957.800000,4.700000,8.900000,16.300000,6.400000,-3.400000,0.800000,46.069444
8,2017-01-09,46.069444,39.500000,106.000000,14.000000,26.000000,0.000000,959.800000,955.900000,4.700000,9.200000,17.300000,6.600000,-4.100000,0.300000,50.986111
9,2017-01-10,50.986111,49.000000,102.000000,19.000000,99.000000,0.000000,957.500000,950.100000,7.200000,5.200000,12.400000,4.400000,-3.500000,1.100000,23.701389


In [2]:
#Creamos conjuntos de datos
X = datos.iloc[:,1:-1].values
y = datos.iloc[:,-1].values

In [6]:
#Dividir concunto de entrenamiento y de testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [15]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Error promedio: ' + str(np.mean(errors)))
    print('Precision: ' + str(accuracy))
    return accuracy

In [21]:
from sklearn.ensemble import RandomForestRegressor
#Error cuadrado medio como criterio de division por defecto
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0) 
regressor.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [18]:
resultado = evaluate(regressor, X_test, y_test)

Error promedio: 5.752439247896856
Precision: 75.22997703831028


In [22]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [24]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  8.0min finished
C:\Python\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [25]:
rf_random.best_params_

{'n_estimators': 1200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': False}

In [30]:
regressor_mod = RandomForestRegressor(n_estimators = 1200, random_state = 0, min_samples_split = 5, min_samples_leaf= 2, 
                                     max_features = 'sqrt', max_depth = 10, bootstrap = False) 
regressor_mod.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=10,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=1200, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [31]:
resultado = evaluate(regressor_mod, X_test, y_test)

Error promedio: 5.689301191506173
Precision: 76.21194761694713
